## Basics

In [1]:
from primitives import Parameters
p = Parameters()
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#%%timeit
from Multiworker_Contract import MultiworkerContract
K=2 #number of tenure levels
mwc=MultiworkerContract(p)
(mwc_J,mwc_W,mwc_Wstar)=mwc.J()

(100,)
Ji shape (7, 11, 11, 100)
W1i shape (7, 11, 11, 100, 2)


c:\Users\andre\Codes\Multi-worker firm\Multiworker_wage_2steps\valuefunction_multi.py:74: RuntimeWarning: divide by zero encountered in scalar divide
  self.rsqr[iz, in0, in1] = res2.fun / np.power(J1[iz, in0, in1, :],2).mean()
c:\Users\andre\Codes\Multi-worker firm\Multiworker_wage_2steps\valuefunction_multi.py:74: RuntimeWarning: invalid value encountered in scalar divide
  self.rsqr[iz, in0, in1] = res2.fun / np.power(J1[iz, in0, in1, :],2).mean()


ValueError: setting an array element with a sequence.

In [31]:
import numpy as np

# Define the shape of the matrix W
shape = (3, 4, 5)  # Example shape (3, 4, 5)

# Create an array where each element equals its last dimension index
# Initialize an array with zeros of the desired shape
W = np.zeros(shape)

# Iterate over the indices of the last dimension and assign them to W
for idx in np.ndindex(W.shape):
    W[idx] = idx[-1]

print("Matrix W:")
print(W)

Matrix W:
[[[0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]]

 [[0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]]

 [[0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]
  [0. 1. 2. 3. 4.]]]


In [9]:
import numpy as np

# Define the shape of the matrix W
shape_W = (3, 5, 7, 4)  # Example shape: 3x5x7x4 matrix

# Create indices for 2nd and 4th dimensions
index_2nd = np.arange(shape_W[1])[np.newaxis,:, np.newaxis, np.newaxis]
index_4th = np.arange(shape_W[3])[np.newaxis, np.newaxis, np.newaxis, :]

# Calculate matrix W where each element is the sum of indices
W = index_2nd + index_4th

# Example of accessing specific elements
print("W[0, 1, 0, 3] =", W[0, 1, 0, 3])  # Output: 1 + 3 = 4
print("W[2, 4, 0, 2] =", W[2, 4, 0, 2])  # Output: 4 + 2 = 6

W[0, 1, 0, 3] = 4


IndexError: index 2 is out of bounds for axis 0 with size 1

1

In [5]:
mwc.w_grid[99]*(mwc.sum_size[0,0,10,10,0,0])
#mwc.sum_wage[0,10,10,10,99,99] 
#Same value of maximum wage at all the maximum sizes. Keep in mind that this ignores the wage and size at the very first step.

1010.9748841899266